<img src="https://teaching.bowyer.io/SDSAI/0/img/IMPERIAL_logo_RGB_Blue_2024.svg" alt="Imperial Logo" width="500"/><br /><br />

Machine Learning Foundations and Data Preparation
==============
### SURG70098 - Surgical Data Science and AI
### Stuart Bowyer

## Intended Learning Outcomes
1.  Describe what is meant by machine learning and the fundamental aspects of machine learning methods 
1.  Perform exploratory data analyses (EDA) of given datasets to understand how they can be used for machine learning
1.  Prepare datasets for use in machine learning by cleaning, transforming, and integrating them


## Session Outline
1.  [Machine Learning Foundations](#pandas)
1.  [Exploratory Data Analysis](#pandas)
1.  [Data Cleaning](#pandas)
1.  [Data Transformation](#pandas)
1.  [Data Integration](#pandas)
1.  [Wrap Up](#wrap_up)

# Connecting to MIMIC
*   We will connect to the MIMIC databases using the `pandas-gbq` library
*   You should have already got access to MIMIC and setup a Google Cloud project before the session
*   To make the examples in this session work:
    *   Go to you [BigQuery project](https://console.cloud.google.com/bigquery) to get your `project_id`
    *   Input your `project_id` below, in my case it is `mimic-project-439312`
    *   Run this code block and it will store the datasets in your environment for use later on

In [209]:
%pip install pandas_gbq

import pandas as pd
import pandas_gbq

project_id = "mimic-project-439314"  # <--- INPUT YOUR PROJECT ID HERE

df_ventilator_setting = pandas_gbq.read_gbq("""
  SELECT *
  FROM `physionet-data.mimiciv_derived.ventilator_setting`
  WHERE ventilator_type IS NOT NULL
  LIMIT 1000
""", project_id=project_id)

df_vitalsign = pandas_gbq.read_gbq("""
  SELECT *
  FROM `physionet-data.mimiciv_derived.vitalsign`
  LIMIT 10000
""", project_id=project_id)


# Machine Learning Foundations
TODO

# Exploratory Data Analysis

# Data Cleaning
Data cleaning is concerned with taking messy real-world data and making is appropriate for use in a model

## What Makes Data 'Messy'
Based on what you know about programming with data, what issues might there with using the following data to build a model?

| PatientID | Name         | Age   | Weight       | Height      | BloodPressure | Temp     | Diagnosis       | Gender  | AppointmentDate  |
|-----------|--------------|-------|--------------|-------------|---------------|----------|-----------------|---------|------------------|
| 101       | John         | 45    | 180 lbs      | 6'2"        | 120/80        | 98.6 F   | Hypertension    | M       | 2024-09-15       |
| 102       | Jane         |       | 70kg         | 165 cm      | 130/85        | 36.5 C   | Diabetes        | Female  | 15th Sep 2024    |
| 103       | Mike         | 53    | 250 lbs      | 72 in       | 140/90        | 100 F    | Hypertension    | M       | 2024-09-14       |
| 104       | Anna         | 401   | 50.5 kgs     | 5'4"        |               | 37 C     | Asthma          | F       | 09/16/2024       |
| 105       | Emily        | 60    | 130          | 160cm       |               |          | None            | Female  | 2024-09-13       |
| 106       | Chris        | -12   | 80kg         |             | 120/75        | 99.1 F   | Asthma          | Male    | 2024-09-12       |
| 107       | Sarah        | 40    | 165.3        | 5.5 ft      | 115/75        |          | Hypertension    | F       | September 14, 24 |
| 108       | Bob          | 67    | 200 lbs      |             |               |          | Heart Disease   | Male    | 2024/09/15       |
| 109       | Laura        | 32    | 140 lbs      | 1.70m       | 120/80        | 99F      | Hypertension    | Female  | 2024-09-14       |
| 110       | David        | 75    | 90kg         | 180 cm      | 110/70        | 37.5C    | None            | M       | 9/15/24          |


### Some Answers...
*   Units are inconsistent within the columns
*   Many missing values
*   Date formatting is inconsistent
*   Outliers (obvious in age)
*   Categorisations are inconsistent

| PatientID | Name         | Age   | Weight       | Height      | BloodPressure | Temp     | Diagnosis       | Gender  | AppointmentDate  |
|-----------|--------------|-------|--------------|-------------|---------------|----------|-----------------|---------|------------------|
| 101       | John         | 45    | 180 lbs      | 6'2"        | 120/80        | 98.6 F   | Hypertension    | M       | 2024-09-15       |
| 102       | Jane         |       | 70kg         | 165 cm      | 130/85        | 36.5 C   | Diabetes        | Female  | 15th Sep 2024    |
| 103       | Mike         | 53    | 250 lbs      | 72 in       | 140/90        | 100 F    | Hypertension    | M       | 2024-09-14       |
| 104       | Anna         | 401   | 50.5 kgs     | 5'4"        |               | 37 C     | Asthma          | F       | 09/16/2024       |
| 105       | Emily        | 60    | 130          | 160cm       |               |          | None            | Female  | 2024-09-13       |
| 106       | Chris        | -12   | 80kg         |             | 120/75        | 99.1 F   | Asthma          | Male    | 2024-09-12       |
| 107       | Sarah        | 40    | 165.3        | 5.5 ft      | 115/75        |          | Hypertension    | F       | September 14, 24 |
| 108       | Bob          | 67    | 200 lbs      |             |               |          | Heart Disease   | Male    | 2024/09/15       |
| 109       | Laura        | 32    | 140 lbs      | 1.70m       | 120/80        | 99F      | Hypertension    | Female  | 2024-09-14       |
| 110       | David        | 75    | 90kg         | 180 cm      | 110/70        | 37.5C    | None            | M       | 9/15/24          |

**What effect do you think these 'issues' will have on a model built from this data?**

## "Garbage In Garbage Out"

<table style='table-layout: fixed; width: 100%; margin-top: 0;'>
  <tbody>
    <tr>
      <td>
        <img src="https://teaching.bowyer.io/SDSAI/3/img/Garbage_in_garbage_out_chatgpt.webp ">
      </td>
      <td>
        <ul>
          <li>It will probably be clear that any model you build based on poor quality, messy data will produce poor quality results</li>
          <li>'Data Cleaning' is the process of improving your input data by correcting or removing errors and inconsistencies</li>
          <li>The following are some examples of key data cleaning methods; however, there is no limit to the type of data issues you can find in your data</li>
        </ul>
      </td>
    </tr>
  </tbody>
</table>

### Cleaning Medical Data
*   One of the biggest challenges in working with medical data is how 'messy' thay are
*   There are no definitive approaches to resolving any of these issues. You will need to use judgement about the:
    *   data type,
    *   data purpose, and
    *   <strong>importantly</strong> clinical context (you have an opportunity here)

## Missing Data
*   Missing data is one of the most common data issues you will find with medical data
*   Some common reasons for missing data are:
    *   Human factors (entry error, non-response, loss)
    *   Systemic factors (changing practice/guidelines, limited machine output)
    *   Availability (patient opt-out, research approvals, linkage, cost-of-acquisition)

❓ **Can you suggest how we might resolve missing data?**


| PatientID | Name         | Age   | Weight       | Height      | BloodPressure | Temp     | Diagnosis       | Gender  | AppointmentDate  |
|-----------|--------------|-------|--------------|-------------|---------------|----------|-----------------|---------|------------------|
| 102       | Jane         |       | 70kg         | 165 cm      | 130/85        | 36.5 C   | Diabetes        | Female  | 15th Sep 2024    |
| 104       | Anna         | 401   | 50.5 kgs     | 5'4"        |               | 37 C     | Asthma          | F       | 09/16/2024       |
| 105       | Emily        | 60    | 130          | 160cm       |               |          | None            | Female  | 2024-09-13       |
| 106       | Chris        | -12   | 80kg         |             | 120/75        | 99.1 F   | Asthma          | Male    | 2024-09-12       |
| 107       | Sarah        | 40    | 165.3        | 5.5 ft      | 115/75        |          | Hypertension    | F       | September 14, 24 |
| 108       | Bob          | 67    | 200 lbs      |             |               |          | Heart Disease   | Male    | 2024/09/15       |

#### Identifying Missing Data in pandas
*   The standard way to define missing data in pandas is with special symbols `NaN` or `None`
*   To find these, there is a pandas method `.isna()`
*   There is also the `.notna()` to find which elements are not missing

In [38]:
df_messy = pd.read_csv("https://teaching.bowyer.io/SDSAI/3/data/messy_data.csv")
df_messy.isna()

,PatientID,Name,Age,Weight,Height,BloodPressure,Temp,Diagnosis,Gender,AppointmentDate
0,False,False,False,False,False,False,False,False,False,False
1,False,False,True,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,True,False,False,False,False
4,False,False,False,False,False,True,True,True,False,False
5,False,False,False,False,True,False,False,False,False,False
6,False,False,False,False,False,False,True,False,False,False
7,False,False,False,False,True,True,True,False,False,False
8,False,False,False,False,False,False,False,False,False,False
9,False,False,False,False,False,False,False,True,False,False


#### Counting Missing Data in pandas
*   You can then find the number missing, per column, with `.sum()`

In [39]:
df_messy.isna().sum()

PatientID          0
Name               0
Age                1
Weight             0
Height             2
BloodPressure      3
Temp               3
Diagnosis          2
Gender             0
AppointmentDate    0
dtype: int64

### Dropping Missing Values

*   **What:** Exclude the missing data points from your data set
*   **How:** If the downstream model permits, you can drop an individual value, otherwise either drop the whole row or column
*   **Advantages:** This ensures your analysis/model/etc is based on the most accurate/representative data
*   **Disadvantages:** You often need to remove either the whole row or column, which wastes potentially useful data elements

#### Dropping Missing Data in pandas
*   To drop a row with missing data, pandas has the `.dropna()` method
*   Here you can specify which columns to look for missing data

In [56]:
df_messy.dropna(subset='BloodPressure')

*   To drop a column with missing data, you can simply specify the column you wish to drop by name

In [41]:
df_messy.drop('BloodPressure', axis=1)

### Imputation (Replacing Missing Values)
*   **What:** Imputation is the process of replacing a missing value with a substitute
*   **How:** A range of functions can be used to generate a substitute, from averages to predictive
*   **Advantages:** This maximises the available dataset
*   **Disadvantages:** Value substitution introduces inaccuracy and uncertainty into the data

#### Imputation Methods
##### Constant Value
*   Substitute missing values with a fixed constant (based on some context/knowledge)
*   ```df['A'].fillna(0)```

#### Average Value
*   Substutute missing values with the mean value of the non-missing values
*   ```df['B'].fillna(df['B'].mean())```
*   Could also be the `median()` or `mode()`

#### Random (Hot-deck)
*   Substitute missing values with a random selection from the available values
*   `rnd = pd.Series(np.random.choice(df['C'].dropna(), size=len(df['C'])))`
*   `df['C'].fillna(rnd)`

#### Regression
*   Substitute missing values with a predicted value based on a regression model of other (non-missing) variables

#### K-Nearest Neighbours
*   Substitute missing values with a predicted value based on a similarity model to other observations

#### Which Method to Use
Your choice of which imputation method to use will be based on:
*   **Domain knowledge** - what do the data represent, what assumptions are valid
*   **Data type** - numerical or categorical
*   **Amount of data** - 'small' data will probably limit accuracy of complex approaches
*   **Amount missing** - simple subsututions for a high proportion of missing data will introduce bias
*   **Downstream model requirements** - are models liable to overfitting

### ❓ Quick Exercise
Spend the next 5 minutes exploring/addressing the missing data in the `df_ventilator_setting` MIMIC dataset

Hints
*   Start by identifying how many values per observation (i.e. per column) are missing
*   Try using each of the methods we have covered
*   Consider the data types, as some are not compatible

## Miss-Typed Data
*   It is very common for medical data to have been imported as a text string; however, data can often have their type changed
*   As a general rule, you want your types to be as simple as possible while maintaining the underlying meaning, i.e.:
    *   free text (`str`) → `float` → categorical (`str`) → `int` → `bool`
*   You should be careful of using integers to represent categorical data though, as it can imply ordinality that does not exist

### Type Conversions in pandas
*   We covered pandas type conversions extensively in lecture 2
*   Types can be easily modified in pandas with the `.astype()` or `pd.to_numeric()` methods
*   The `str` methods also allow for conversion of free text inputs to more constrained types

In [72]:
df_bp = df_messy[['BloodPressure']].dropna()
df_bp[['Systolic','Diastolic']] = df_bp['BloodPressure'] \
    .str.split('/', expand=True) \
    .astype(int)

df_bp


,BloodPressure,Systolic,Diastolic
0,120/80,120,80
1,130/85,130,85
2,140/90,140,90
5,120/75,120,75
6,115/75,115,75
8,120/80,120,80
9,110/70,110,70


## Inconsistent Data
*   Inconsistent data are those where different observations in a given column do not represent the same thing
*   A simple example would be measurements with different units (weight in kg or lbs) or taken in different ways (temperature taken internally or peripherally)
*   Data should be consistently represented to be useful for modelling and machine learning
*   Inconsistent data can be either:
    *   Converted to a common consistent format (e.g. converting temperatures to common units `degrees C`)
    *   Split into distinct observations (e.g. splitting to `internal_temp` and `peripheral_temp` observations)

### ❓ Quick Exercise
Spend the next 5 minutes cleaning types and inconsistencies in the `Temp`, `Gender`, and `BloodGlucose` columns of the toy dataset in `df_messy`

Hints
*   The conversion from temp in 'F' to 'C' is `(Temp°F − 32) × 5/9 = Temp°C`
*   Also look at the `BGMethod` column when considering the `BloodGlucose` values

In [ ]:
df_messy = pd.read_csv("https://teaching.bowyer.io/SDSAI/3/data/messy_data.csv")

# YOUR CODE HERE

## Outliers / Invalid Values
*   Outliers and invalid values (i.e. impossible/non-physiological values) can be introduced to data by human, acquisition, and transfer errors, as well as exceptional inputs
*   You can attempt to define outliers statistically; however, you should always start by considering the context
    *   i.e. can the value be negative, or greater than 100%, etc.

### Z-score Based Outlier Removal
*   If your data are normally (Gaussian) distributed z-scores can be used to identify outliers
*   The z-score is a number that represents how many 'standard deviations' a value is from the mean
*   Therefore, by setting a limit on the permissible z-score, values 'too far' from average can be removed
*   The z-score for a value can be calculated as follows:
$$Z = \frac{X - \mu}{\sigma}$$

where: $X$ is the value, $\mu$ is the mean of the column, $\sigma$ is the standard deviation of the column

<img src="https://teaching.bowyer.io/SDSAI/3/img/z-score_Illustration.png" alt="z-score illustration" width="700"/>

#### Z-score Based Outlier Removal Example
*   The `temperature` values in the `vitalsign` MIMIC-IV dataset are approximately normal
*   With the following, we can eliminate values that are more than 3 standard deviations from the mean

In [237]:
# Cast to numeric values
df_outlier = pd.DataFrame()
df_outlier['temp'] = pd.to_numeric(df_vitalsign['temperature'])

# Store original temp for comparison at the end
df_outlier['old_temp'] = df_outlier['temp']

# Compute z-scores using previous equation
df_outlier['zscore'] = (df_outlier['temp'] - df_outlier['temp'].mean()) / df_outlier['temp'].std()

# Replace extreme values with NaN by using the mask function
df_outlier['temp'] = df_outlier['temp'].mask(df_outlier['zscore'].abs() > 3.0)

# Check the result
df_outlier[['old_temp','temp']].describe()


,old_temp,temp
count,1732.000000,1727.000000
mean,36.895612,36.894823
std,0.459767,0.454323
min,35.500000,35.560000
25%,36.610000,36.610000
50%,36.890000,36.890000
75%,37.170000,37.170000
max,38.280000,38.220000


### Interquartile Range (IQR) Based Outlier Removal
*   When your data are not normally distributed, the IQR can be used as it does not assume a specific distribution
*   The IQR is the range between the 25th and 75th percentiles in the data
*   Similarly to the z-scores, you can identify outliers by how many times the IQR they are from the first and third quartiles

<img src="https://teaching.bowyer.io/SDSAI/3/img/IQR_Illustration.png" alt="IQR illustration" width="700"/>


### Interquartile Range (IQR) Based Outlier Removal Example
*   The `glucose` values in the `vitalsign` MIMIC-IV dataset are non-normal
*   With the following, we can eliminate values that are more than 1.5 IQR from IQR

In [245]:
# Cast to numeric values
df_outlier = pd.DataFrame()
df_outlier['glucose'] = pd.to_numeric(df_vitalsign['glucose'])

# Store original temp for comparison at the end
df_outlier['old_glucose'] = df_outlier['glucose']

# Compute IQR
q1 = df_outlier['glucose'].quantile(0.25)
q3 = df_outlier['glucose'].quantile(0.75)
iqr = q3 - q1

# Compute limits
lower_limit = q1 - 1.5 * iqr
upper_limit = q3 + 1.5 * iqr

# Replace extreme values with NaN by using the mask function
df_outlier['glucose'] = df_outlier['glucose'].mask(
    (df_outlier['glucose'] < lower_limit) | (df_outlier['glucose'] > upper_limit))

# Check the result
df_outlier[['old_glucose','glucose']].describe()

,old_glucose,glucose
count,220.000000,211.000000
mean,149.947727,141.481043
std,60.651774,43.870883
min,62.000000,62.000000
25%,108.000000,107.500000
50%,136.500000,134.000000
75%,175.250000,163.500000
max,466.000000,267.000000


## ❓ Exercise 3.2 - Data Cleaning
Clean as much of the `df_messy` data frame as you can. Copy from the previous exercises/examples where useful.

Some of the columns in the data do not have an obvious way to impute them, you can use your judgement what to do with these.

In [ ]:
df_messy = pd.read_csv("https://teaching.bowyer.io/SDSAI/3/data/messy_data.csv")

# YOUR CODE HERE

# Data Transformation

## Data Normalisation

## Data Encoding

## Feature Engineering

# Data Integration

## Merging Data Sets

## Aligning Data Sets
*   e.g. sample time, data types, 

# Wrap Up

## Before Next Session
*   Read chapters 12, 13, and 15 of the 'Secondary Analysis of Electronic Health Records' book (on the reading list)